# Continous variables

If the number of states is continous, $x\in[0,x_m]$, position. 

For a truck, there are positions $x,y$ angles $\theta$, and velocities $\dot{x}$. Then the state vecotr is 
$$
s = 
\begin{bmatrix}
x \\
y \\
\theta \\
\dot{x} \\
\dot{y} \\
\dot{\theta} 
\end{bmatrix}
$$
the values for which are contonous. For a helicoptcer, there is also $z$ psoiton, role, pitch, yaw (for orientation). Also correposinding rate of changes. These comprise the state vector 
$$
s = 
\begin{bmatrix}
x \\
y \\
z \\
\phi \\
\theta \\
\omega \\
\dot{x} \\
\dot{y} \\
\dot{z} \\
\dot{\phi} \\
\dot{\theta} \\
\dot{\omega}
\end{bmatrix}
$$
This are states for _continous state Markov decision process_.  

#### Lunar landing application

COntrol thrusters. The actions are 
- do nothing (fall under the force of gravity)
- left thruster
- main thruster
- right thrister

The state vector here resds,
$$
s = 
\begin{bmatrix}
x \\
y \\
\dot{x} \\
\dot{y} \\
\theta \\
\dot{\theta} \\
\dot{l} \\
\dot{r} 
\end{bmatrix}
$$
where $\dot{\theta}$ is the angular velocity, $l$ is the position of the left leg on the ground, $r$ is the position of the right leg on the ground (these two are binary, 1, or 0). 


Reward parameters:  
- The reword function for laning is $100$ $140$. 
- Additional reward for moving toward/away from pad.  
- Crash: $-100$
- Soft landing $+100$  
- Leg grounded $+10$
- Fire main engine $-0.3$
- Fire side thrusters $-0.03$

> Goal is to learn the policy $\pi$, that given the state vector $s$, pciks the action $a=\pi(s)$ so as to maximize the return. 

Train the neuroal to compute/approximate the state action value function $Q$ of $s$. This will result in a choice of a good action $a$. 

We consider a NN, that takes the current state, the current action and computes/approximates $Q(s)$. So the input to NN is the state vector $s$ and the action $a$, encoded with _one-hot_ method $[1,0,0,0]$ for nothing, $[0,1,0,0]$ for left, $[0,0,1,0]$ for right, $[0,0,0,1]$ for main and $a = [0,0,0,0,1]$ for left. So we column-stack the $s$ and $a$. This is input $\vec{x}=[s,a]^T$, the output is $y=Q(s,a)$. 

Contrary to the supervised learning, we input here a paer $\vec{x},y$ and try to get $Q(s,a)$. 
The method is to use _Belman Equation_ to create lots of learning exampls.  Then, use supervise learning as usual, to earn a mapping from a state-axtion pair $s,a$ to the $Q(s,a)$.  

Computing the training set using Bellman equation:
$$
Q(x=\{s,a\}) = y; \text{ where } y= R(s) + \gamma \times \max_{a'}(Q(s',a'))
$$
We need to learn function 
$f_{w,b}(x)=y$
so we need a lot of training examples. 
This is done by randomly choosing actions, and computing a list of tuples 
$(s,a,R(s),s')$.  

Consider $\Big(s^{(1)},a^{(1)},R(s^{(1)}),s^{'(1)}\Big)$. 

To make the tuple we fill the table as $x^{(i)} = f(s^{(i)},a^{(i)})$ from the Bellman equatins, we get $y^{(i)}=f(R(s^{(i)}),s^{'(i)})$

| x | y |
| --- | --- |
| (s^{(i)},a^{(i)}) | y^{(i)}|

The $Q$ function is unknown but can be computed from random guess. 

| x | y |
| --- | --- |
| $x^{(i)} = (s^{(i)}, a^{(i)})$ | $y^{(i)} = R(s^{(i)}) + \gamma\times\max_{a'}Q(s^{'(i)},a')$ |
| $x^{(i+1)} = (s^{(i+1)}, a^{(i+1)})$ | $y^{(i+1)} = R(s^{(i+1)}) + \gamma\times\max_{a'}Q(s^{'(i+1)},a')$ |

This gives a training set can is used to train NN. 

#### Full algorithm (DQN algorithm)
1. Start with a NN with random guesses for $Q(s,a)$
2. Take a random action and get $\Big(s^{(1)},a^{(1)},R(s^{(1)}),s^{'(1)}\Big)$. 
3. Store $N=10000$ most recent examples, which are called _Replay Buffer_ 
4. Train the NN using the $\{x,y\}$ pairs where $x=(s,a)$ and $y=(R(s)+\gamma\times\max_{a'}Q(s',a'))$. Where the last $10^4$ are stored. Note, we start with random $Q()$ function. The result of the NN is $Q_{\rm new} \approx y$. Then, update $Q=Q_{\rm new}$. 

#### Efficient arcitecture for DQN 

The original NN is, given 12 numbers for numbers and each action (nothing, left, main, right is one-hot encoded in the vector). 

$\vec{x} = [s,a]^T$ $\rightarrow$ $64$ $\rightarrow$ $64$ $\rightarrow$ $1$ $\rightarrow$ $Q(s,a)$.  

This is inefficinet as the NN has to trained each time for each action separately.  
A more eficient NN is with input of $8$ number and $4$ unit output for each action:

$\vec{x} = [s,a]^T$ $\rightarrow$ $64$ $\rightarrow$ $64$ $\rightarrow$ $4$ $\rightarrow$ $[Q(s,\text{nothing}),Q(s,\text{left}),Q(s,\text{main}),Q(s,\text{right})]^T$.  

Now, only one inference is required. Also, now calculation of $R(s)+\gamma\times\max_{a'}Q(s',a')$ is more efficinet, as it is computed for all actions at the same time. 

#### Algorithm refinement: $\epsilon$-greedy policy

In learning how to approximate $Q(s,a)$, actions $a$ still have to be taken. It is non-trivial to find, how to chouse these actions. This is done with **$\epsilon$-greedy algorithm**. There are two options to do so.  

Option 1: 
- Pick an action $a$ that maximsizes current $Q(s,a)$

Option 2: **$\epsilon$-greedy apolicy** ($\epsilon=0.05$):
- With probability $0.95$ pick the action that maximises $Q(s,a)$ _Exploitation step_.
- With probability $0.05$ pick the action randomly. _Exploration step_.

Randomness is helpfull, when the initial guess is heavily biased towards one specific action and the algorithm gets stuck with a subset of actions. In other workds it helps overcome "reconceptions". 

It is common to start with high $\epsilon$ and reduce it during the learning. $\epsilon: 1.\rightarrow 0.01$. 

The algorithms in reinforcment learning are very picky in terms of the choice of the hyperparameters. One has to be careful. 

#### Refinments: Mini-batches

Mini-batches is used in reinformcnet and supervised learning. 

- In supervised learning it is like the following:

Mini-batches helps when the size of the dataset is very large. In standard gradient descend the updates need to be run over the enire dataset (recall $\sum_{i=1}^m(f_{(w,b)}(x^{(i)})-y^{(i)}))^2$ term). 
In mini-batches, only a subset of examples is considered as  $\sum_{i=1}^{n_i}(f_{(w,b)}(x^{(i)})-y^{(i)}))^2$ where $n_i$ is the size of the subset or a batch. 

The "full-batch" gradient descend goes to the minimum of the function reliably. nini-batch algorithm, however, because of downsampled data, goes to the minimum _noisily_ and _chaotically_, but less computationally expesive. 

- In reinforcment learning it is like the following: 

The reply buffer is split in subsets (sub-buffers) and use it to train the NN. This speeds-up the training. 

#### Refinment: Soft-update

Recall, that when we train the NN in reinforment learning, we start with a random guess for the $Q$ function and update it to the $Q_{\rm new}$ after each training run. However, this update can lead to a drastic change in the algorithm and it is not neceseraly an inprovement. E.g., unlucky steps are possible. Then, consider 

$Q=Q_{\rm new}$, e.g., $w\rightarrow w_{\rm new}$ and $b\rightarrow b_{\rm new}$. 

In the soft-update method, we keep old and new model parameters as following: 

$w = 0.01 w_{\rm new} + 0.99 w$ and $b = 0.01 b_{\rm new} + 0.99 b$. With $0.99$ is a new hyperparameter, that controls how soft is the update. 

Such algorithm converges more reliably. 

#### State of the reinforment learning



